In [1]:
import os
import sys
import time
import pandas as pd
import arcpy
from arcgis.features import GeoAccessor, GeoSeriesAccessor

In [2]:
parcels = r"E:\Data\REMM_Parcels_with_TAZID.shp"
parcels_sdf = pd.DataFrame.spatial.from_featureclass(parcels)
parcels_sdf.columns

Index(['FID', 'Join_Count', 'TARGET_FID', 'parcel_id', 'CO_TAZID', 'DEVACRES',
       'TAZID', 'CO_FIPS', 'CO_NAME', 'SHAPE'],
      dtype='object')

In [3]:
taz = '.\\Inputs\\TAZ_WFRC_UTM12.shp'
taz_sdf = pd.DataFrame.spatial.from_featureclass(taz)
taz_sdf.columns

Index(['FID', 'CO_TAZID', 'DEVACRES', 'TAZID', 'CO_FIPS', 'CO_NAME', 'SHAPE'], dtype='object')

In [4]:
# new buildings csv
nb_2015 = pd.read_csv(r"E:\Projects\REMM2_For_Python3_MAX_SPRAWL\REMMRun\run225year2015newbuildings.csv")
nb_2020 = pd.read_csv(r"E:\Projects\REMM2_For_Python3_MAX_SPRAWL\REMMRun\run225year2020newbuildings.csv")
nb_2030 = pd.read_csv(r"E:\Projects\REMM2_For_Python3_MAX_SPRAWL\REMMRun\run225year2030newbuildings.csv")
nb_2040 = pd.read_csv(r"E:\Projects\REMM2_For_Python3_MAX_SPRAWL\REMMRun\run225year2040newbuildings.csv")
nb_2050 = pd.read_csv(r"E:\Projects\REMM2_For_Python3_MAX_SPRAWL\REMMRun\run225year2050newbuildings.csv")

In [5]:
nb_by_parcel_2015 = pd.DataFrame(nb_2015['parcel_id'].value_counts()).reset_index()
nb_by_parcel_2015.columns = ['parcel_id', 'count_nb']
nb_by_parcel_2015.head(5)

,parcel_id,count_nb
0,698367,1
1,410912,1
2,765219,1
3,439590,1
4,812328,1


In [6]:
nb_by_parcel_2020 = pd.DataFrame(nb_2020['parcel_id'].value_counts()).reset_index()
nb_by_parcel_2020.columns = ['parcel_id', 'count_nb']

nb_by_parcel_2030 = pd.DataFrame(nb_2030['parcel_id'].value_counts()).reset_index()
nb_by_parcel_2030.columns = ['parcel_id', 'count_nb']

nb_by_parcel_2040 = pd.DataFrame(nb_2040['parcel_id'].value_counts()).reset_index()
nb_by_parcel_2040.columns = ['parcel_id', 'count_nb']

nb_by_parcel_2050 = pd.DataFrame(nb_2050['parcel_id'].value_counts()).reset_index()
nb_by_parcel_2050.columns = ['parcel_id', 'count_nb']

In [7]:
parcels_2015 = parcels_sdf.merge(nb_by_parcel_2015, left_on='parcel_id', right_on='parcel_id', how='left')
parcels_2020 = parcels_sdf.merge(nb_by_parcel_2020, left_on='parcel_id', right_on='parcel_id', how='left')
parcels_2030 = parcels_sdf.merge(nb_by_parcel_2030, left_on='parcel_id', right_on='parcel_id', how='left')
parcels_2040 = parcels_sdf.merge(nb_by_parcel_2040, left_on='parcel_id', right_on='parcel_id', how='left')
parcels_2050 = parcels_sdf.merge(nb_by_parcel_2050, left_on='parcel_id', right_on='parcel_id', how='left')

In [8]:
nb_by_taz_2015 = pd.DataFrame(parcels_2015.groupby('TAZID', as_index=False)['count_nb'].sum())
nb_by_taz_2015.head(5)

,TAZID,count_nb
0,141,0.0
1,142,0.0
2,143,0.0
3,144,0.0
4,145,0.0


In [9]:
nb_by_taz_2020 = pd.DataFrame(parcels_2020.groupby('TAZID', as_index=False)['count_nb'].sum())
nb_by_taz_2020.columns = ['TAZID', 'count_nb']

nb_by_taz_2030 = pd.DataFrame(parcels_2030.groupby('TAZID', as_index=False)['count_nb'].sum())
nb_by_taz_2030.columns = ['TAZID', 'count_nb']

nb_by_taz_2040 = pd.DataFrame(parcels_2040.groupby('TAZID', as_index=False)['count_nb'].sum())
nb_by_taz_2040.columns = ['TAZID', 'count_nb']

nb_by_taz_2050 = pd.DataFrame(parcels_2050.groupby('TAZID', as_index=False)['count_nb'].sum())
nb_by_taz_2050.columns = ['TAZID', 'count_nb']

In [10]:
nb_by_taz_2015['count_nb'].sum()

248.0

In [11]:
taz_2015 = taz_sdf.merge(nb_by_taz_2015, left_on='TAZID', right_on='TAZID', how='left')
taz_2015['count_nb'].fillna(0, inplace=True)

taz_2020 = taz_sdf.merge(nb_by_taz_2020, left_on='TAZID', right_on='TAZID', how='left')
taz_2020['count_nb'].fillna(0, inplace=True)

taz_2030 = taz_sdf.merge(nb_by_taz_2030, left_on='TAZID', right_on='TAZID', how='left')
taz_2030['count_nb'].fillna(0, inplace=True)

taz_2040 = taz_sdf.merge(nb_by_taz_2040, left_on='TAZID', right_on='TAZID', how='left')
taz_2040['count_nb'].fillna(0, inplace=True)

taz_2050 = taz_sdf.merge(nb_by_taz_2050, left_on='TAZID', right_on='TAZID', how='left')
taz_2050['count_nb'].fillna(0, inplace=True)

In [12]:
# Create Total buildings dataframe
taz_9999 = taz_2015.copy()
taz_9999['count_nb'] = (taz_2015['count_nb'] + taz_2020['count_nb'] + taz_2030['count_nb'] + 
                        taz_2040['count_nb'] + taz_2050['count_nb'])
taz_9999.head()

,FID,CO_TAZID,DEVACRES,TAZID,CO_FIPS,CO_NAME,SHAPE,count_nb
0,0,491896,194.00,1896,49,UTAH,"{'rings': [[[413322.9699998261, 4468422.370000...",0.0
1,1,492490,163.53,2490,49,UTAH,"{'rings': [[[446538.3899997908, 4447316.290000...",8.0
2,2,492493,172.52,2493,49,UTAH,"{'rings': [[[445717.82294877776, 4447316.87087...",16.0
3,3,492632,316.82,2632,49,UTAH,"{'rings': [[[442594.8061128018, 4445568.069968...",28.0
4,4,491859,1391.86,1859,49,UTAH,"{'rings': [[[406997.88032511633, 4470180.33657...",0.0


In [13]:
# create output gdb
outputs = '.\\Outputs'
gdb = os.path.join(outputs, "new_buildings.gdb")
if not arcpy.Exists(gdb):
    arcpy.CreateFileGDB_management(outputs, "new_buildings.gdb")

In [17]:
taz_2015["year"]= 2015
taz_2020["year"]= 2020
taz_2030["year"]= 2030
taz_2040["year"]= 2040
taz_2050["year"]= 2050
taz_9999["year"]= 9999

#concatenate all the tables together to one file
taz_tables = [taz_2015, taz_2020, taz_2030, taz_2040, taz_2050, taz_9999]
taz_all_sdf = pd.concat(taz_tables)
taz_all_sdf.spatial.to_featureclass(location=os.path.join(gdb, 'new_buildings_maxsprawl'))

'E:\\Projects\\REMM-Analysis-Tools\\Max_Sprawl\\Outputs\\new_buildings.gdb\\new_buildings_maxsprawl'